# For fine-tune

## Install requirements

In [1]:
from IPython.display import clear_output

In [7]:
!pip install yfinance
!pip install fix_yahoo_finance
!pip install pymoo
clear_output()

In [8]:
# Load Packages
import numpy as np
import pandas as pd
from pandas_datareader import data
import matplotlib.pyplot as plt
%matplotlib inline

## Fine tune data

In [10]:
# Import yfinance package
import yfinance as yf

# Set the start and end date
start_date = '2020-04-30' #'2020-04-30'
end_date = '2024-04-30'   #'2024-04-30'

ticker = ['AAPL', 'NVDA', 'MSFT', 'AMZN', 'META', 'GOOGL', 'TSLA', 'AVGO', 'GOOG', 'BRK-B',
          'JPM', 'LLY', 'V', 'XOM', 'UNH', 'MA', 'COST', 'WMT', 'HD', 'PG',
          'NFLX', 'JNJ', 'ABBV', 'CRM', 'BAC', 'ORCL', 'MRK', 'CVX', 'WFC', 'KO',
          'CSCO', 'ACN', 'NOW', 'TMO', 'MCD', 'IBM', 'AMD', 'DIS', 'PEP', 'LIN',
          'ABT', 'ISRG', 'PM', 'GE', 'ADBE', 'GS', 'QCOM', 'TXN', 'CAT', 'INTU']

# Get the data
fine_tune_data = yf.download(ticker, start_date, end_date, auto_adjust=False) # adding auto_adjust=False to get Adj Close

[*********************100%***********************]  50 of 50 completed


In [11]:
fine_tune_data = fine_tune_data['Adj Close']
fine_tune_data

import numpy as np
# Log of percentage change
cov_matrix = fine_tune_data.pct_change().apply(lambda x: np.log(1+x)).cov()
cov_matrix

# Yearly returns for individual companies
ind_er = fine_tune_data.resample('YE').last().pct_change().mean()
ind_er

mu = np.array(ind_er)
cov = np.array(cov_matrix)

### Run the experiment HV

In [15]:
### Test HV index

In [19]:
# import libs
from pymoo.core.problem import ElementwiseProblem
from pymoo.core.repair import Repair
from pymoo.algorithms.moo.sms import SMSEMOA
from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.optimize import minimize
import operator

class PortfolioProblem(ElementwiseProblem):
    def __init__(self, mu, cov, risk_free_rate=0.02, **kwargs):
        super().__init__(n_var=len(np.array(ticker)), n_obj=2, xl=0.0, xu=1.0, **kwargs)
        self.mu = mu
        self.cov = cov
        self.risk_free_rate = risk_free_rate

    def _evaluate(self, x, out, *args, **kwargs):
        exp_return = x @ self.mu
        exp_risk = np.sqrt(x.T @ self.cov @ x) * np.sqrt(250.0)
        sharpe = (exp_return - self.risk_free_rate) / exp_risk

        out["F"] = [exp_risk, -exp_return]
        out["sharpe"] = sharpe

In [21]:
class PortfolioRepair(Repair):
    def _do(self, problem, X, **kwargs):
        X[X < 1e-3] = 0
        return X / X.sum(axis=1, keepdims=True)

In [23]:
import numpy as np
from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.operators.crossover.sbx import SBX
from pymoo.operators.crossover.pntx import TwoPointCrossover
from pymoo.operators.crossover.ux import UniformCrossover
from pymoo.operators.sampling.rnd import FloatRandomSampling, BinaryRandomSampling
from pymoo.operators.sampling.lhs import LHS
from pymoo.optimize import minimize
from pymoo.indicators.hv import HV

# Define configurations combining sampling and crossover operators
configurations = [
    {'name': 'BinaryRandom_SBX',   'sampling': BinaryRandomSampling(), 'crossover': SBX()},
    {'name': 'BinaryRandom_TwoPt', 'sampling': BinaryRandomSampling(), 'crossover': TwoPointCrossover()},
    {'name': 'BinaryRandom_Uniform','sampling': BinaryRandomSampling(), 'crossover': UniformCrossover()},

    {'name': 'FloatRandom_SBX',    'sampling': FloatRandomSampling(),  'crossover': SBX()},
    {'name': 'FloatRandom_TwoPt',  'sampling': FloatRandomSampling(),  'crossover': TwoPointCrossover()},
    {'name': 'FloatRandom_Uniform','sampling': FloatRandomSampling(),  'crossover': UniformCrossover()},

    {'name': 'LHS_SBX',            'sampling': LHS(),               'crossover': SBX()},
    {'name': 'LHS_TwoPt',          'sampling': LHS(),               'crossover': TwoPointCrossover()},
    {'name': 'LHS_Uniform',        'sampling': LHS(),               'crossover': UniformCrossover()},
]

In [25]:
import matplotlib.pyplot as plt

def calculate_HV(F):
    """Calculate hypervolume of the front F against a fixed reference point."""
    ref_point = np.array([0.5, -0.1])
    ind = HV(ref_point=ref_point)

    # plt.scatter(F[:, 0], F[:, 1], facecolor="none", edgecolors="blue", alpha=0.5, label="Pareto-Optimal Portfolio")
    # plt.scatter(ref_point[0], ref_point[1], facecolor="none", edgecolors="green", alpha=0.5, label="reference point")
    # plt.legend()
    # plt.xlabel("expected volatility")
    # plt.ylabel("expected return")
    # plt.show()
   
    return ind(F)

In [27]:
# Number of independent runs
test_time = 10
# Prepare storage for HV results
test_results = {conf['name']: {'500': [], '600': []} for conf in configurations}

for run_idx in range(test_time):
    # Assuming these functions are defined elsewhere:    
    problem = PortfolioProblem(mu, cov)    

    for config in configurations:
        # Build the NSGA2 algorithm with chosen operators
        algorithm = NSGA2(
            pop_size=200,
            sampling=config['sampling'],
            crossover=config['crossover'],
            repair=PortfolioRepair()
        )

        # 500 generations
        print(f"Run {run_idx+1}/{test_time} - {config['name']} - 500 gens")
        res_500 = minimize(
            problem,
            algorithm,
            termination=('n_gen', 500),
            seed=run_idx,
            verbose=False
        )

        X, F, sharpe = res_500.opt.get("X", "F", "sharpe")
        # F = F * [1, -1]  # Adjust for minimization/maximization
    
        hv_500 = calculate_HV(F)
        test_results[config['name']]['500'].append(hv_500)
        print(f"  Hypervolume@500: {hv_500:.6f}")

        # 600 generations (reuse same algorithm instance or rebuild)
        print(f"Run {run_idx+1}/{test_time} - {config['name']} - 600 gens")
        res_600 = minimize(
            problem,
            algorithm,
            termination=('n_gen', 600),
            seed=run_idx,
            verbose=False
        )
        
        X, F, sharpe = res_600.opt.get("X", "F", "sharpe")
        # F = F * [1, -1]  # Adjust for minimization/maximization    
        
        hv_600 = calculate_HV(F)
        test_results[config['name']]['600'].append(hv_600)
        print(f"  Hypervolume@600: {hv_600:.6f}\n")

# After all runs, you can summarize results
def summarize_results(results_dict):
    summary = {}
    for name, gens in results_dict.items():
        summary[name] = {
            'mean_HV_500': np.mean(gens['500']),
            'std_HV_500': np.std(gens['500']),
            'mean_HV_600': np.mean(gens['600']),
            'std_HV_600': np.std(gens['600'])
        }
    return summary

if __name__ == '__main__':
    summary = summarize_results(test_results)
    for name, stats in summary.items():
        print(f"{name}: 500 gens -> mean {stats['mean_HV_500']:.6f}, std {stats['std_HV_500']:.6f}; \
              600 gens -> mean {stats['mean_HV_600']:.6f}, std {stats['std_HV_600']:.6f}")

Run 1/10 - BinaryRandom_SBX - 500 gens


KeyboardInterrupt: 

## Run the experiment gain money

In [30]:
# import libs
from pymoo.core.problem import ElementwiseProblem
from pymoo.core.repair import Repair
from pymoo.algorithms.moo.sms import SMSEMOA
from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.optimize import minimize
import operator

In [32]:
class PortfolioProblem(ElementwiseProblem):
    def __init__(self, mu, cov, returns_df, risk_free_rate=0.02, VaR_confidence = 0.95, hhi_limit=0.15, **kwargs):
        super().__init__(n_var=len(np.array(ticker)), n_obj=2, xl=0.0, xu=1.0, **kwargs)
        self.mu = mu
        self.cov = cov
        self.returns_df = returns_df
        self.risk_free_rate = risk_free_rate
        self.VaR_confidence = VaR_confidence
        self.hhi_limit = hhi_limit        
        self.DAYS_PER_YEAR = 250
        self.z_alpha = norm.ppf(self.VaR_confidence)

    def _evaluate(self, x, out, *args, **kwargs):
        x = np.asarray(x).flatten()
        
        exp_return = x @ self.mu
        volatility_daily = np.sqrt(x @ self.cov @ x)
        volatility_annual = volatility_daily * np.sqrt(self.DAYS_PER_YEAR)
        exp_return_annual = exp_return * self.DAYS_PER_YEAR

        # sharp ratio
        sharpe = (exp_return - self.risk_free_rate) / volatility_annual

        port_daily_ret = self.returns_df @ x #每日報酬
        
        '''
        # --- Calmar ratio ---
        
        # 若沒有回測區間太短會失敗，使用者自行確認 returns_df 的長度
        cum = np.cumprod(1 + port_daily_ret)
        running_max = np.maximum.accumulate(cum)
        running_max[running_max == 0] = 1e-6
        drawdowns = (running_max - cum) / running_max
        max_dd = drawdowns.max() if len(drawdowns) else 0.0
        calmar = np.inf if max_dd == 0 else exp_return_annual / max_dd
        '''
        
        # --- Herfindahl–Hirschman Index (HHI) 作多元化 ---
        hhi = np.sum(np.square(x))          # ∑ w_i²，1 = 全押單一資產
        diversity = 1 - hhi                 # 值愈大愈分散
        
        # --- Value at Risk ---
        var = -np.percentile(port_daily_ret, (1 - self.VaR_confidence) * 100)
        
        
        
        out["F"] = [volatility_annual, -exp_return]
        out["sharpe"] = sharpe
        # out["calmar"] = calmar
        out["diversity"] = diversity
        out["var"] = var

In [34]:
class PortfolioRepair(Repair):
    def _do(self, problem, X, **kwargs):
        X[X < 1e-3] = 0
        return X / X.sum(axis=1, keepdims=True)

In [36]:
from pymoo.indicators.hv import HV
from scipy.stats import norm

def calculate_expected_returns(current_data):
    ind_er = current_data.resample('YE').last().pct_change().mean()
    # print(ind_er)
    return ind_er

def calculate_covariance_matrix(current_data):
    cov_matrix = current_data.pct_change().apply(lambda x: np.log(1+x)).cov()
    # print(cov_matrix)
    return cov_matrix

def calculate_returns_df_matrix(current_data):
    returns_df = current_data.pct_change().dropna()
    return returns_df

def record_stock_price(stock_alloc, current_budget, price_day):
    # extract trade date
    # print(f'current budget is {current_budget[0]}')  
    trade_date = fine_tune_data.index[price_day]
    # print(f'record stock price is {trade_date}')

    # extract stock price in trade date
    prices_series = fine_tune_data.loc[[trade_date]]
    prices_array = prices_series.to_numpy().ravel()  # shape (n,)
    # print(f'Prices: {prices_array}')
    # print(f'Stock allocation: {stock_alloc}')    
    
    current_budget[0] = stock_alloc @ prices_array  
    
    # print(f'current budget is {current_budget[0]}')  

def execute_trades(stock_alloc, final_weight, current_budget, price_day):
    # extract trade date
    # print(f'current budget is {current_budget[0]}')
    trade_date = fine_tune_data.index[price_day]
    # print(f'trade data is {trade_date}')

    # extract stock price in trade date
    prices_series = fine_tune_data.loc[[trade_date]]
    prices_array = prices_series.to_numpy().ravel()  # shape (n,)
    # print(f'Prices: {prices_array}')
    # print(f'Stock allocation: {stock_alloc}')
    
    if not np.allclose(stock_alloc, np.zeros(len(ticker))):
        # print("you are here attention")
        current_budget[0] = stock_alloc @ prices_array   
    
    dollar_alloc = current_budget[0]*final_weight   

    # Element-wise division to get number of shares
    stock_alloc[:] = dollar_alloc / prices_array  # modifies caller's array
    # print(f'Stock allocation (shares): {stock_alloc}')

    
    # print(f'stock_alloc is {stock_alloc}')
    # print(f'current budget is {current_budget[0]}')

In [38]:
data_size = len(fine_tune_data)
test_time = 10  # Independent runs
test_period = 250
VaR_confidence = 0.95
hhi_limit = 0.25

# Storage
test_sharpe_results = {conf['name']: [] for conf in configurations}
test_diversity_results = {conf['name']: [] for conf in configurations}
test_min_var_results = {conf['name']: [] for conf in configurations}

for run_idx in range(test_time):
    print(f"Run {run_idx + 1}/{test_time}")

    for conf in configurations:    

        sharpe_current_budget = [1]
        sharpe_current_budget[0] = 1_000_000
        sharpe_stock_alloc = np.zeros(len(ticker)) 
         
        diversity_current_budget = [1]
        diversity_current_budget[0] = 1_000_000
        diversity_stock_alloc = np.zeros(len(ticker))  
        
        min_var_current_budget = [1]
        min_var_current_budget[0] = 1_000_000
        min_var_stock_alloc = np.zeros(len(ticker))  
        
        name = conf['name']
        print(f"{name}")
        
        for day in range(test_period):    
            print(f"Current day is: {day}")
            serach_day = data_size - (test_period-day)
            current_data = fine_tune_data.iloc[:serach_day] 

            if(day%20 != 0):  
                print("you are doing sharpe")
                record_stock_price(sharpe_stock_alloc, sharpe_current_budget, serach_day)

                print("you are doing diversity")
                record_stock_price(diversity_stock_alloc, diversity_current_budget, serach_day)

                # print("you are doing max_var")
                # record_stock_price(max_var_stock_alloc, max_var_current_budget, serach_day, max_var_budget_history, max_var_stock_alloc_history)

                print("you are doing min_var")
                record_stock_price(min_var_stock_alloc, min_var_current_budget, serach_day)
                continue
        
            mu = calculate_expected_returns(current_data)
            cov = calculate_covariance_matrix(current_data)
            returns_df = calculate_returns_df_matrix(current_data)
            problem = PortfolioProblem(mu, cov, returns_df, risk_free_rate=0.02, VaR_confidence = VaR_confidence, hhi_limit=hhi_limit)

            algorithm = NSGA2(pop_size=200,
                    sampling=conf['sampling'],
                    crossover=conf['crossover'],
                    repair=PortfolioRepair())

            res = minimize( problem,
                    algorithm,
                    ('n_gen', 500),
                    seed=run_idx,
                    verbose=False) 
    
            # Get optimal portfolio weights
            X, F, sharpe, diversity, var = res.opt.get("X", "F", "sharpe", "diversity", "var")        
          
            # Select portfolio with maximum Sharpe ratio
            max_sharpe_idx = sharpe.argmax()            
            max_diversity_idx = diversity.argmax()
            # max_var_idx = var.argmax()
            min_var_idx = var.argmin()            
    
            sharpe_optimal_weights = X[max_sharpe_idx]            
            diversity_optimal_weights = X[max_diversity_idx]
            # max_var_optimal_weights = X[max_var_idx]
            min_var_optimal_weights = X[min_var_idx] 

            execute_trades(sharpe_stock_alloc, sharpe_optimal_weights, sharpe_current_budget, serach_day)  # Assume this function exists  
            execute_trades(diversity_stock_alloc, diversity_optimal_weights, diversity_current_budget, serach_day)  # Assume this function exists   
            # execute_trades(max_var_stock_alloc, max_var_optimal_weights, max_var_current_budget, serach_day)  # Assume this function exists    
            execute_trades(min_var_stock_alloc, min_var_optimal_weights, min_var_current_budget, serach_day)  # Assume this function exists

        name = conf['name']
        test_sharpe_results[name].append(sharpe_current_budget[0])
        test_diversity_results[name].append(diversity_current_budget[0])
        test_min_var_results[name].append(min_var_current_budget[0])

Run 1/10
BinaryRandom_SBX
Current day is: 0


KeyboardInterrupt: 

In [40]:
def summarize_results(results_dict):
    summary = {}
    for name, budget_list in results_dict.items():
        summary[name] = {
            'mean': np.mean(budget_list),
            'std': np.std(budget_list)
        }
    return summary

if __name__ == '__main__':
    print(test_sharpe_results)
    print(test_diversity_results)
    print(test_min_var_results)
    sharpe_summary = summarize_results(test_sharpe_results)
    diversity_summary = summarize_results(test_diversity_results)
    min_var_summary = summarize_results(test_min_var_results)

    print("== Sharpe Summary ==")
    for name, stats in sharpe_summary.items():
        print(f"{name}: mean sharpe = {stats['mean']:.6f}, std = {stats['std']:.6f}")

    print("\n== Diversity Summary ==")
    for name, stats in diversity_summary.items():
        print(f"{name}: mean diversity = {stats['mean']:.6f}, std = {stats['std']:.6f}")

    print("\n== Min Var Summary ==")
    for name, stats in min_var_summary.items():
        print(f"{name}: mean min var = {stats['mean']:.6f}, std = {stats['std']:.6f}")

{'BinaryRandom_SBX': [], 'BinaryRandom_TwoPt': [], 'BinaryRandom_Uniform': [], 'FloatRandom_SBX': [], 'FloatRandom_TwoPt': [], 'FloatRandom_Uniform': [], 'LHS_SBX': [], 'LHS_TwoPt': [], 'LHS_Uniform': []}
{'BinaryRandom_SBX': [], 'BinaryRandom_TwoPt': [], 'BinaryRandom_Uniform': [], 'FloatRandom_SBX': [], 'FloatRandom_TwoPt': [], 'FloatRandom_Uniform': [], 'LHS_SBX': [], 'LHS_TwoPt': [], 'LHS_Uniform': []}
{'BinaryRandom_SBX': [], 'BinaryRandom_TwoPt': [], 'BinaryRandom_Uniform': [], 'FloatRandom_SBX': [], 'FloatRandom_TwoPt': [], 'FloatRandom_Uniform': [], 'LHS_SBX': [], 'LHS_TwoPt': [], 'LHS_Uniform': []}
== Sharpe Summary ==
BinaryRandom_SBX: mean sharpe = nan, std = nan
BinaryRandom_TwoPt: mean sharpe = nan, std = nan
BinaryRandom_Uniform: mean sharpe = nan, std = nan
FloatRandom_SBX: mean sharpe = nan, std = nan
FloatRandom_TwoPt: mean sharpe = nan, std = nan
FloatRandom_Uniform: mean sharpe = nan, std = nan
LHS_SBX: mean sharpe = nan, std = nan
LHS_TwoPt: mean sharpe = nan, std =

C:\Users\kaoyoung\anaconda3\Lib\site-packages\numpy\core\fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\kaoyoung\anaconda3\Lib\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
C:\Users\kaoyoung\anaconda3\Lib\site-packages\numpy\core\_methods.py:206: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
C:\Users\kaoyoung\anaconda3\Lib\site-packages\numpy\core\_methods.py:163: RuntimeWarning: invalid value encountered in divide
  arrmean = um.true_divide(arrmean, div, out=arrmean,
C:\Users\kaoyoung\anaconda3\Lib\site-packages\numpy\core\_methods.py:198: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


# Solve problem with NSGA_II

## Install requirment

In [74]:
from IPython.display import clear_output

!pip install yfinance
!pip install fix_yahoo_finance
!pip install pymoo
clear_output()

# Load Packages
import numpy as np
import pandas as pd
from pandas_datareader import data
import matplotlib.pyplot as plt
%matplotlib inline

In [75]:
# import libs
from pymoo.core.problem import ElementwiseProblem
from pymoo.core.repair import Repair
from pymoo.algorithms.moo.sms import SMSEMOA
from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.optimize import minimize
import operator

## Get the data

In [85]:
# Import yfinance package
import yfinance as yf

# Set the start and end date
start_date = '2020-04-30' #'2020-04-30'
end_date = '2025-04-30'   #'2025-04-30'

ticker = ['AAPL', 'NVDA', 'MSFT', 'AMZN', 'META', 'GOOGL', 'TSLA', 'AVGO', 'GOOG', 'BRK-B',
          'JPM', 'LLY', 'V', 'XOM', 'UNH', 'MA', 'COST', 'WMT', 'HD', 'PG',
          'NFLX', 'JNJ', 'ABBV', 'CRM', 'BAC', 'ORCL', 'MRK', 'CVX', 'WFC', 'KO',
          'CSCO', 'ACN', 'NOW', 'TMO', 'MCD', 'IBM', 'AMD', 'DIS', 'PEP', 'LIN',
          'ABT', 'ISRG', 'PM', 'GE', 'ADBE', 'GS', 'QCOM', 'TXN', 'CAT', 'INTU']

# Get the data
data = yf.download(ticker, start_date, end_date, auto_adjust=False) # adding auto_adjust=False to get Adj Close

[*********************100%***********************]  50 of 50 completed


In [87]:
data = data['Adj Close']
# data

## NSGA2 SETTING

In [92]:
class PortfolioProblem(ElementwiseProblem):
    def __init__(self, mu, cov, returns_df, risk_free_rate=0.02, VaR_confidence = 0.95, hhi_limit=0.15, **kwargs):
        super().__init__(n_var=len(np.array(ticker)), n_obj=2, xl=0.0, xu=1.0, **kwargs)
        self.mu = mu
        self.cov = cov
        self.returns_df = returns_df
        self.risk_free_rate = risk_free_rate
        self.VaR_confidence = VaR_confidence
        self.hhi_limit = hhi_limit        
        self.DAYS_PER_YEAR = 250
        self.z_alpha = norm.ppf(self.VaR_confidence)

    def _evaluate(self, x, out, *args, **kwargs):
        x = np.asarray(x).flatten()
        
        exp_return = x @ self.mu
        volatility_daily = np.sqrt(x @ self.cov @ x)
        volatility_annual = volatility_daily * np.sqrt(self.DAYS_PER_YEAR)
        exp_return_annual = exp_return * self.DAYS_PER_YEAR

        # sharp ratio
        sharpe = (exp_return - self.risk_free_rate) / volatility_annual

        port_daily_ret = self.returns_df @ x #每日報酬
        
        '''
        # --- Calmar ratio ---
        
        # 若沒有回測區間太短會失敗，使用者自行確認 returns_df 的長度
        cum = np.cumprod(1 + port_daily_ret)
        running_max = np.maximum.accumulate(cum)
        running_max[running_max == 0] = 1e-6
        drawdowns = (running_max - cum) / running_max
        max_dd = drawdowns.max() if len(drawdowns) else 0.0
        calmar = np.inf if max_dd == 0 else exp_return_annual / max_dd
        '''
        
        # --- Herfindahl–Hirschman Index (HHI) 作多元化 ---
        hhi = np.sum(np.square(x))          # ∑ w_i²，1 = 全押單一資產
        diversity = 1 - hhi                 # 值愈大愈分散
        
        # --- Value at Risk ---
        var = -np.percentile(port_daily_ret, (1 - self.VaR_confidence) * 100)
        
        
        
        out["F"] = [volatility_annual, -exp_return]
        out["sharpe"] = sharpe
        # out["calmar"] = calmar
        out["diversity"] = diversity
        out["var"] = var

In [94]:
class PortfolioRepair(Repair):

    def _do(self, problem, X, **kwargs):
        X[X < 1e-3] = 0
        return X / X.sum(axis=1, keepdims=True)

## Needed function for running 

In [138]:
from pymoo.indicators.hv import HV
from scipy.stats import norm

def calculate_expected_returns(current_data):
    ind_er = current_data.resample('YE').last().pct_change().mean()
    # print(ind_er)
    return ind_er

def calculate_covariance_matrix(current_data):
    cov_matrix = current_data.pct_change().apply(lambda x: np.log(1+x)).cov()
    # print(cov_matrix)
    return cov_matrix

def calculate_returns_df_matrix(current_data):
    returns_df = current_data.pct_change().dropna()
    return returns_df

def record_stock_price(stock_alloc, current_budget, price_day, budget_history, stock_alloc_history):
    # extract trade date
    # print(f'current budget is {current_budget[0]}')  
    trade_date = data.index[price_day]
    # print(f'record stock price is {trade_date}')

    # extract stock price in trade date
    prices_series = data.loc[[trade_date]]
    prices_array = prices_series.to_numpy().ravel()  # shape (n,)
    # print(f'Prices: {prices_array}')
    # print(f'Stock allocation: {stock_alloc}')    
    
    current_budget[0] = stock_alloc @ prices_array  
    
    budget_history.append(current_budget[0])
    # stock_alloc_history.append(stock_alloc.copy())
    # print(f'stock_alloc is {stock_alloc}')
    # print(f'current budget is {current_budget[0]}')  
        
def execute_trades(stock_alloc, final_weight, current_budget, price_day, budget_history, stock_alloc_history):
    # extract trade date
    # print(f'current budget is {current_budget[0]}')
    trade_date = data.index[price_day]
    # print(f'trade data is {trade_date}')

    # extract stock price in trade date
    prices_series = data.loc[[trade_date]]
    prices_array = prices_series.to_numpy().ravel()  # shape (n,)
    # print(f'Prices: {prices_array}')

    # print(f'Stock allocation: {stock_alloc}')

    
    if not np.allclose(stock_alloc, np.zeros(len(ticker))):
        current_budget[0] = stock_alloc @ prices_array
    
    
    
    dollar_alloc = current_budget[0]*final_weight
        

    # Element-wise division to get number of shares
    stock_alloc[:] = dollar_alloc / prices_array  # modifies caller's array
    # print(f'Stock allocation (shares): {stock_alloc}')

    budget_history.append(current_budget[0])
    stock_alloc_history.append(stock_alloc.copy())
    # print(f'stock_alloc is {stock_alloc}')
    # print(f'current budget is {current_budget[0]}')
    

## Run the whole process

In [141]:
'''
%%time
for config in configurations:
    print(f"\nRunning configuration: {config['name']}")
'''

results = {}
sharpe_portfolio_history = []
# calmar_portfolio_history = []
diversity_portfolio_history = []
# max_var_portfolio_history = []
min_var_portfolio_history = []
data_size = len(data)

sharpe_current_budget = [1]
sharpe_current_budget [0] = 1_000_000
sharpe_stock_alloc = np.zeros(len(ticker))  
# calmar_current_budget = 1_000_000
# calmar_stock_alloc = np.zeros(len(ticker))  
diversity_current_budget = [1]
diversity_current_budget[0] = 1_000_000
diversity_stock_alloc = np.zeros(len(ticker))  
# max_var_current_budget = 1_000_000
# max_var_stock_alloc = np.zeros(len(ticker))  
min_var_current_budget = [1]
min_var_current_budget[0] = 1_000_000
min_var_stock_alloc = np.zeros(len(ticker))  

sharpe_budget_history = []
sharpe_stock_alloc_history = []
# calmar_budget_history = []
# calmar_stock_alloc_history = []
diversity_budget_history = []
diversity_stock_alloc_history = []
# max_var_budget_history = []
# max_var_stock_alloc_history = []
min_var_budget_history = []
min_var_stock_alloc_history = []

test_period = 250

VaR_confidence = 0.95
hhi_limit = 0.25
# 這裡的 hhi_limit 會影響到多元化的程度，hhi_limit 越小，越分散

for day in range(test_period):    
    print(f"Current day is: {day}")
    serach_day = data_size - (test_period-day)
    current_data = data.iloc[:serach_day] 

    if(day%20 != 0): 
        # print("you are doing sharpe")
        record_stock_price(sharpe_stock_alloc, sharpe_current_budget, serach_day, sharpe_budget_history, sharpe_stock_alloc_history)

        # print("you are doing diversity")
        record_stock_price(diversity_stock_alloc, diversity_current_budget, serach_day, diversity_budget_history, diversity_stock_alloc_history)

        # print("you are doing max_var")
        # record_stock_price(max_var_stock_alloc, max_var_current_budget, serach_day, max_var_budget_history, max_var_stock_alloc_history)

        # print("you are doing min_var")
        record_stock_price(min_var_stock_alloc, min_var_current_budget, serach_day, min_var_budget_history, min_var_stock_alloc_history)
        continue

    # print(current_data)
    mu = calculate_expected_returns(current_data)
    cov = calculate_covariance_matrix(current_data)
    returns_df = calculate_returns_df_matrix(current_data)
    problem = PortfolioProblem(mu, cov, returns_df, risk_free_rate=0.02, VaR_confidence = VaR_confidence, hhi_limit=hhi_limit)
    # print(current_data)
    
    '''
    algorithm = NSGA2(repair=PortfolioRepair(),
                      sampling = config['sampling'],
                      crossover=config['crossover'])
    '''
    
    algorithm = NSGA2(repair=PortfolioRepair())
    
    
    res = minimize(problem,
                   algorithm,
                   seed=1,
                   verbose=False) 

    # Get optimal portfolio weights
    X, F, sharpe, diversity, var = res.opt.get("X", "F", "sharpe", "diversity", "var")
    F = F * [1, -1]  # Adjust for minimization/maximization    
    

    # Select portfolio with maximum Sharpe ratio
    max_sharpe_idx = sharpe.argmax()
    # max_calmar_idx = calmar.argmax()
    max_diversity_idx = diversity.argmax()
    # max_var_idx = var.argmax()
    min_var_idx = var.argmin()

    '''
    plt.scatter(F[:, 0], F[:, 1], facecolor="none", edgecolors="blue", alpha=0.5, label="Pareto-Optimal Portfolio")
    # plt.scatter(cov.diagonal() ** 0.5 * np.sqrt(250.0), mu, facecolor="none", edgecolors="black", s=30, label="Asset")
    plt.scatter(F[max_sharpe_idx, 0], F[max_sharpe_idx, 1], marker="x", s=100, color="red", label="Max Sharpe Portfolio")
    # plt.scatter(F[max_sharpe, 0], F[max_sharpe, 1], marker="x", s=100, color="red", label="Max Sharpe Portfolio")
    plt.scatter(F[max_diversity_idx, 0], F[max_diversity_idx, 1], marker="x", s=100, color="brown", label="Max Diversity Portfolio")
    plt.scatter(F[max_var_idx, 0], F[max_var_idx, 1], marker="x", s=100, color="yellow", label="Max Var Portfolio")
    plt.scatter(F[min_var_idx, 0], F[min_var_idx, 1], marker="x", s=100, color="black", label="Min Var Portfolio")
    plt.scatter(np.sqrt(x.T @ cov @ x) * np.sqrt(250.0), x.T @ mu, marker="*", s=100, color="green", label="Equal weights")
    plt.legend()
    plt.xlabel("expected volatility")
    plt.ylabel("expected return")
    plt.show()
    '''
    
    sharpe_optimal_weights = X[max_sharpe_idx]
    # calmar_optimal_weights = X[max_calmar_idx]
    diversity_optimal_weights = X[max_diversity_idx]
    # max_var_optimal_weights = X[max_var_idx]
    min_var_optimal_weights = X[min_var_idx]

    # calculate HV
    # HVindex = calculate_HV(F)

    '''
    # Add other indicators (f1, ..., fk) for daily allocation
    additional_indicators = calculate_technical_indicators(current_data)  # Assume this function exists
    final_weights = adjust_weights_with_indicators(optimal_weights, additional_indicators)
    '''


    
    # Make investment with adjusted weights
    # print("you are doing sharpe")
    execute_trades(sharpe_stock_alloc, sharpe_optimal_weights, sharpe_current_budget, serach_day, sharpe_budget_history, sharpe_stock_alloc_history)  # Assume this function exists
    

    # print("you are doing calmar")
    # calmar_daily_return = calmar_execute_trades(calmar_stock_alloc, calmar_optimal_weights, calmar_current_budget, serach_day, calmar_budget_history, calmar_stock_alloc_history)  # Assume this function exists
    # calmar_portfolio_history.append(calmar_daily_return)

    # print("you are doing diversity")
    execute_trades(diversity_stock_alloc, diversity_optimal_weights, diversity_current_budget, serach_day, diversity_budget_history, diversity_stock_alloc_history)  # Assume this function exists
    
    # print("you are doing max_var")
    # execute_trades(max_var_stock_alloc, max_var_optimal_weights, max_var_current_budget, serach_day, max_var_budget_history, max_var_stock_alloc_history)  # Assume this function exists

    # print("you are doing min_var")
    execute_trades(min_var_stock_alloc, min_var_optimal_weights, min_var_current_budget, serach_day, min_var_budget_history, min_var_stock_alloc_history)  # Assume this function exists

Current day is: 0
Current day is: 1
Current day is: 2
Current day is: 3
Current day is: 4
Current day is: 5
Current day is: 6
Current day is: 7
Current day is: 8
Current day is: 9
Current day is: 10
Current day is: 11
Current day is: 12
Current day is: 13
Current day is: 14
Current day is: 15
Current day is: 16
Current day is: 17
Current day is: 18
Current day is: 19


In [143]:


print("--------------------")
print(sharpe_budget_history)
print("--------------------")
print(sharpe_stock_alloc_history)
print("--------------------")
print(diversity_budget_history)
print("--------------------")
print(diversity_stock_alloc_history)
print("--------------------")
print(min_var_budget_history)
print("--------------------")
print(min_var_stock_alloc_history)


--------------------
[1000000, 1002395.0121452613, 965814.9244596425, 900857.7518014247, 903450.8725343274, 889776.3721427877, 960872.7985253334, 926172.3055717151, 948445.1751178526, 961213.107593413, 965902.9111314354, 940200.9680130507, 970432.8292661373, 948831.2512328003, 967268.8437108097, 979429.7787581644, 1001585.3160028935, 1018543.8258649954, 1012417.4952355445, 1017085.5787975774]
--------------------
[array([   0.        ,  240.92559869,    0.        ,    0.        ,
          0.        ,    0.        ,    0.        ,    0.        ,
          0.        ,    0.        ,    0.        ,    0.        ,
          0.        ,    0.        ,    0.        ,    0.        ,
          0.        ,    0.        ,    0.        ,    0.        ,
          0.        ,    5.11149799,    0.        ,    0.        ,
          0.        ,    0.        ,    0.        ,    0.        ,
        254.1353677 ,    0.        ,    0.        ,    0.        ,
         26.48152864,    0.        ,    0.    

#### for i in range(len(stock_alloc_history)):
    print(stock_alloc_history[i]) 

In [45]:
# Plotting
plt.figure(figsize=(14, 6))
plt.plot(sharpe_budget_history, label="sharpe portfolio Value", color='blue')
#plt.plot(calmar_budget_history, label="calmar Portfolio Value", color='red')
plt.plot(diversity_budget_history, label="diversity portfolio Value", color='green')
plt.plot(max_var_budget_history, label="max var portfolio Value", color='peru')
plt.plot(min_var_budget_history, label="min var portfolio Value", color='yellow')
plt.title("Portfolio Value Over Time")
plt.xlabel("Time (unit: 10 Days)")
plt.ylabel("Value")
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()

NameError: name 'sharpe_budget_history' is not defined

In [47]:
# Compare results
for name, res in results.items():
    print(f"\nThe result of {name} is as follows\n")
    X, F, sharpe = res.opt.get("X", "F", "sharpe")
    F = F * [1, -1]
    max_sharpe = sharpe.argmax()

    '''
    # equal weights
    x = np.zeros((len(ticker), 1))
    x = x + 1.0/len(x)

    plt.scatter(F[:, 0], F[:, 1], facecolor="none", edgecolors="blue", alpha=0.5, label="Pareto-Optimal Portfolio")
    plt.scatter(cov.diagonal() ** 0.5 * np.sqrt(250.0), mu, facecolor="none", edgecolors="black", s=30, label="Asset")
    plt.scatter(F[max_sharpe, 0], F[max_sharpe, 1], marker="x", s=100, color="red", label="Max Sharpe Portfolio")
    plt.scatter(np.sqrt(x.T @ cov @ x) * np.sqrt(250.0), x.T @ mu, marker="*", s=100, color="green", label="Equal weights")
    plt.legend()
    plt.xlabel("expected volatility")
    plt.ylabel("expected return")
    plt.show()

    plt.scatter(F[:, 0], F[:, 1], facecolor="none", edgecolors="blue", alpha=0.5, label="Pareto-Optimal Portfolio")
    plt.scatter(F[max_sharpe, 0], F[max_sharpe, 1], marker="x", s=100, color="red", label="Max Sharpe Portfolio")
    plt.legend()
    plt.xlabel("expected volatility")
    plt.ylabel("expected return")
    plt.show()
    '''
    x = X[max_sharpe].T

    print("Best Sharpe: \nReturn     = ", x.T @ mu)
    print("Volatility = ", np.sqrt(x.T @ cov @ x) * np.sqrt(250.0))
    print("sharpo ratio = ", (x.T @ mu - 0.02)/(np.sqrt(x.T @ cov @ x) * np.sqrt(250.0)))


In [49]:
X, F, sharpe = res.opt.get("X", "F", "sharpe")
F = F * [1, -1]
max_sharpe = sharpe.argmax()

# equal weights
x = np.zeros((len(ticker), 1))
x = x + 1.0/len(x)

plt.scatter(F[:, 0], F[:, 1], facecolor="none", edgecolors="blue", alpha=0.5, label="Pareto-Optimal Portfolio")
plt.scatter(cov.diagonal() ** 0.5 * np.sqrt(250.0), mu, facecolor="none", edgecolors="black", s=30, label="Asset")
plt.scatter(F[max_sharpe, 0], F[max_sharpe, 1], marker="x", s=100, color="red", label="Max Sharpe Portfolio")
plt.scatter(np.sqrt(x.T @ cov @ x) * np.sqrt(250.0), x.T @ mu, marker="*", s=100, color="green", label="Equal weights")
plt.legend()
plt.xlabel("expected volatility")
plt.ylabel("expected return")
plt.show()

plt.scatter(F[:, 0], F[:, 1], facecolor="none", edgecolors="blue", alpha=0.5, label="Pareto-Optimal Portfolio")
plt.scatter(F[max_sharpe, 0], F[max_sharpe, 1], marker="x", s=100, color="red", label="Max Sharpe Portfolio")
plt.legend()
plt.xlabel("expected volatility")
plt.ylabel("expected return")
plt.show()

NameError: name 'res' is not defined

In [51]:
allocation = {name: w for name, w in zip(data.columns, X[max_sharpe])}
allocation = sorted(allocation.items(), key=operator.itemgetter(1), reverse=True)

print("Allocation With Best Sharpe")
for name, w in allocation:
    print(f"{name:<5} {w}")

AttributeError: module 'pandas_datareader.data' has no attribute 'columns'

In [53]:
for al in allocation:
    if al[1] <= 1e-2:
        allocation.remove(al)

col_name = []
w1 = []
for name, w in allocation:
    col_name.append(name)
    w1.append(w)
    
fig1, ax1 = plt.subplots()
ax1.pie(w1, labels=col_name, autopct='%1.1f%%',
        shadow=True, startangle=90)
ax1.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.

plt.show()

NameError: name 'allocation' is not defined

In [55]:
x = np.zeros((len(ticker), 1))
x = x + 1.0/len(x)

print("For equal weights: \nReturn     = ", (x.T @ mu)[0])
print("Volatility = ", (np.sqrt(x.T @ cov @ x) * np.sqrt(250.0))[0][0])

For equal weights: 
Return     =  0.11474887960744226
Volatility =  0.194630738676453


In [57]:
x = X[max_sharpe].T

print("Best Sharpe: \nReturn     = ", x.T @ mu)
print("Volatility = ", np.sqrt(x.T @ cov @ x) * np.sqrt(250.0))

NameError: name 'X' is not defined

In [64]:
import numpy as np
from pymoo.algorithms.moo.moead import MOEAD
from pymoo.optimize import minimize
from pymoo.util.ref_dirs import get_reference_directions
from pymoo.core.problem import ElementwiseProblem
from pymoo.core.repair import Repair

class PortfolioProblem(ElementwiseProblem):
    def __init__(self, mu, cov, risk_free_rate=0.02, **kwargs):
        super().__init__(
            n_var=len(mu),
            n_obj=2,
            xl=0.0,
            xu=1.0,
            **kwargs
        )
        self.mu = mu
        self.cov = cov
        self.risk_free_rate = risk_free_rate

    def _evaluate(self, x, out, *args, **kwargs):
        exp_return = x @ self.mu
        exp_risk   = np.sqrt(x.T @ self.cov @ x) * np.sqrt(250.0)
        sharpe     = (exp_return - self.risk_free_rate) / exp_risk
        out["F"]   = [exp_risk, -exp_return]
        out["sharpe"] = sharpe

class PortfolioRepair(Repair):
    def _do(self, problem, X, **kwargs):
        X[X < 1e-3] = 0
        return X / X.sum(axis=1, keepdims=True)

# Compute mu, cov from your data
mu  = calculate_expected_returns(current_data)
cov = calculate_covariance_matrix(current_data)

# Problem definition
problem = PortfolioProblem(mu, cov)

# Reference directions for MOEA/D 
ref_dirs = get_reference_directions("uniform", 2, n_partitions=100)

# MOEA/D algorithm setup
algorithm = MOEAD(
    ref_dirs,
    n_neighbors=20,
    prob_neighbor_mating=0.7,
    repair=PortfolioRepair()
)

# Run optimization
res = minimize(problem, algorithm, seed=1, verbose=False)

# Extract and select optimal portfolio
X, F, sharpe = res.opt.get("X", "F", "sharpe")
F = F * [1, -1]
max_sharpe_idx = sharpe.argmax()
optimal_weights = X[max_sharpe_idx]

# Hypervolume (optional)
HVindex = calculate_HV(F)


C:\Users\kaomi\AppData\Local\Temp\ipykernel_18768\4236682315.py:4: FutureWarning: 'Y' is deprecated and will be removed in a future version, please use 'YE' instead.
  ind_er = current_data.resample('Y').last().pct_change().mean()


HV 0.022752022831910097
